In [154]:
import pandas as pd
import codecs
from sklearn import preprocessing
with codecs.open("data/AllTokyo_202001_202109.csv", "r", "Shift-JIS", "ignore") as file:
    df = pd.read_table(file, delimiter=",")

In [155]:
df.columns
df.dtypes

No              int64
種類             object
地域             object
市区町村コード         int64
都道府県名          object
市区町村名          object
地区名            object
最寄駅：名称         object
最寄駅：距離（分）      object
取引価格（総額）        int64
坪単価           float64
間取り            object
面積（u）          object
取引価格（u単価）     float64
土地の形状          object
間口             object
延床面積（u）        object
建築年            object
建物の構造          object
用途             object
今後の利用目的        object
前面道路：方位        object
前面道路：種類        object
前面道路：幅員（ｍ）    float64
都市計画           object
建ぺい率（％）       float64
容積率（％）        float64
取引時点           object
改装             object
取引の事情等         object
dtype: object

In [156]:
df = df.drop(['No', '都道府県名'], axis=1)

In [157]:
# df[df['用途']=='住宅' and (df['種類']=='中古マンション等' or df['種類']=='宅地(土地と建物)')]
df = df[ (df['用途']=='住宅') &  ( (df['種類']=='中古マンション等') | (df['種類']=='宅地(土地と建物)') )]

In [158]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

### preprocessing columns

In [159]:
df = df.dropna(subset=['建築年', '取引価格（総額）', '延床面積（u）'])
df = df[df['建築年'] != '戦前']

In [160]:
# 昭和元年 : 1926年, 平成元年: 1989, 令和元年: 2019
df['年号'] = df['建築年'].str[:2]

df['和暦_年'] = df['建築年'].str[2:].str.replace('年','').replace('','0').astype(int)
df.loc[df['年号']=='明治','建築年'] = df['和暦_年'] + 1867
df.loc[df['年号']=='大正','建築年'] = df['和暦_年'] + 1911
df.loc[df['年号']=='昭和','建築年'] = df['和暦_年'] + 1925
df.loc[df['年号']=='平成','建築年'] = df['和暦_年'] + 1988
df.loc[df['年号']=='令和','建築年'] = df['和暦_年'] + 2018

In [161]:
# 昭和元年 : 1926年, 平成元年: 1989, 令和元年: 2019
df['取引時点'] = df['取引時点'].str[:4].astype(int) + df['取引時点'].str[6].astype(int) / 4

In [162]:
df.dtypes

種類             object
地域             object
市区町村コード         int64
市区町村名          object
地区名            object
最寄駅：名称         object
最寄駅：距離（分）      object
取引価格（総額）        int64
坪単価           float64
間取り            object
面積（u）          object
取引価格（u単価）     float64
土地の形状          object
間口             object
延床面積（u）        object
建築年            object
建物の構造          object
用途             object
今後の利用目的        object
前面道路：方位        object
前面道路：種類        object
前面道路：幅員（ｍ）    float64
都市計画           object
建ぺい率（％）       float64
容積率（％）        float64
取引時点          float64
改装             object
取引の事情等         object
年号             object
和暦_年            int64
dtype: object

In [173]:
df = df.fillna(0)
df.dtypes

種類            category
地域            category
市区町村コード          int64
市区町村名         category
地区名           category
最寄駅：名称        category
最寄駅：距離（分）     category
取引価格（総額）         int64
坪単価            float64
間取り              int64
面積（u）            int64
取引価格（u単価）      float64
土地の形状         category
間口            category
延床面積（u）       category
建築年              int64
建物の構造         category
用途            category
今後の利用目的       category
前面道路：方位       category
前面道路：種類       category
前面道路：幅員（ｍ）     float64
都市計画          category
建ぺい率（％）        float64
容積率（％）         float64
取引時点           float64
改装               int64
取引の事情等        category
年号            category
和暦_年             int64
dtype: object

In [174]:
num_columns = ['面積（u）']
for column in df.columns:
    if df.dtypes[column] == 'object' and column not in num_columns:
        target_column = df[column].astype(str)
        le = preprocessing.LabelEncoder()
        le.fit(target_column)
        label_encoded_column = le.transform(target_column)
        df[column] = pd.Series(label_encoded_column).astype('category')
for column in num_columns: 
    df[column] = df[column].astype('int64')
df['最寄駅：距離（分）']['30分?60分'] = 30 
df['最寄駅：距離（分）']['1H?1H30'] = 30
df['最寄駅：距離（分）']
df['最寄駅：距離（分）'].astype('int64')

#        == str, '最寄駅：距離（分）'] = 30
    # df['最寄駅：距離（分）'][df['最寄駅：距離（分）'].dtype == str] = 30

24962    0
24964    0
24972    0
24973    0
24974    0
        ..
41727    0
41728    0
41731    0
41732    0
41733    0
Name: 最寄駅：距離（分）, Length: 13342, dtype: int64

In [175]:
df.dtypes

種類            category
地域            category
市区町村コード          int64
市区町村名         category
地区名           category
最寄駅：名称        category
最寄駅：距離（分）     category
取引価格（総額）         int64
坪単価            float64
間取り              int64
面積（u）            int64
取引価格（u単価）      float64
土地の形状         category
間口            category
延床面積（u）       category
建築年              int64
建物の構造         category
用途            category
今後の利用目的       category
前面道路：方位       category
前面道路：種類       category
前面道路：幅員（ｍ）     float64
都市計画          category
建ぺい率（％）        float64
容積率（％）         float64
取引時点           float64
改装               int64
取引の事情等        category
年号            category
和暦_年             int64
dtype: object

In [176]:
# plt.scatter('取引価格（総額）', '建築年', data=df)
# データセットを説明変数と目的変数に分割
X, y = df.drop('取引価格（総額）', axis=1), df['取引価格（総額）']

In [177]:
df = df.dropna()
df

,種類,地域,市区町村コード,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,取引価格（総額）,坪単価,間取り,...,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,年号,和暦_年
24962,0,0,13102,0,0,0,0,42000000,0.0,0,...,0,3.0,0,80.0,600.0,2021.50,0,0,0,43
24964,0,0,13102,0,0,0,0,150000000,0.0,0,...,0,10.0,0,80.0,500.0,2021.25,0,0,0,12
24972,0,0,13102,0,0,0,0,410000000,0.0,0,...,0,5.4,0,80.0,400.0,2021.50,0,0,0,9
24973,0,0,13102,0,0,0,0,35000000,0.0,0,...,0,2.7,0,80.0,400.0,2021.50,0,0,0,39
24974,0,0,13102,0,0,0,0,68000000,0.0,0,...,0,7.0,0,80.0,400.0,2021.50,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41727,0,0,13364,0,0,0,0,4000000,0.0,0,...,0,1.0,0,70.0,200.0,2020.50,0,0,0,47
41728,0,0,13381,0,0,0,0,4500000,0.0,0,...,0,7.5,0,70.0,200.0,2021.75,0,0,0,44
41731,0,0,13401,0,0,0,0,11000000,0.0,0,...,0,0.0,0,70.0,200.0,2021.50,0,0,0,56
41732,0,0,13401,0,0,0,0,12000000,0.0,0,...,0,5.0,0,70.0,200.0,2020.75,0,0,0,57


In [178]:
from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# X_eval, X_test, y_eval, y_test = train_test_split(X_val, y_val, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

In [179]:
!pip install lightgbm

conda 4.10.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
open3d 0.12.0+3483e4b has requirement pyyaml==5.4.1, but you'll have pyyaml 3.13 which is incompatible.
albumentations 1.0.2 has requirement scikit-image>=0.16.1, but you'll have scikit-image 0.15.0 which is incompatible.
You are using pip version 10.0.1, however version 22.0.4 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [180]:
import lightgbm as lgb

from sklearn.model_selection import KFold

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import numpy as np
!pip install --upgrade pandas


# 5つのモデルを保存するリストの初期化
models = []

# 学習用データでの予測値を保存するデータフレームの初期化
df_train_preds = pd.DataFrame({'y_train': y_train})

# 検証用データでの予測値を保存するデータフレームの初期化
df_eval_preds = pd.DataFrame({'y_eval': [],
                              'y_eval_pred': []})

# テストデータでの予測値を保存するデータフレームの初期化
df_test_preds = pd.DataFrame({'y_test': y_test})
# インデックスが0からの連番になるように初期化

df_test_preds.reset_index(inplace=True, drop=True)

# R^2を保存するデータフレームの初期化
df_R2 = pd.DataFrame({'train': [],
                      'eval': [],
                      'test': []})

# RMSEを保存するデータフレームの初期化
df_RMSE =pd.DataFrame({'train': [],
                       'eval': [],
                       'test': []})


# ラウンド数の初期化
round_no = 0


# 【ブロック２：　モデルの学習】
# 学習データの数だけの数列（0行から最終行まで連番）
row_no_list = list(range(len(y_train)))

# KFoldクラスをインスタンス化（これを使って5分割する）
K_fold = KFold(n_splits=10, shuffle=True,  random_state=42)

# KFoldクラスで分割した回数だけ実行（ここでは5回）
for train_cv_no, eval_cv_no in K_fold.split(row_no_list, y_train):
    # ilocで取り出す行を指定
    X_train_cv = X_train.iloc[train_cv_no, :]
    y_train_cv = pd.Series(y_train).iloc[train_cv_no]
    X_eval_cv = X_train.iloc[eval_cv_no, :]
    y_eval_cv = pd.Series(y_train).iloc[eval_cv_no]
    
    # 学習用
    lgb_train = lgb.Dataset(X_train_cv, y_train_cv,
                            free_raw_data=False)
    # 検証用
    lgb_eval = lgb.Dataset(X_eval_cv, y_eval_cv, reference=lgb_train,
                           free_raw_data=False)
    
    # パラメータを設定
    params = {
#         'task': 'train',                # 学習、トレーニング ⇔　予測predict
#               'boosting_type': 'gbdt',        # 勾配ブースティング
              'objective': 'regression',      # 目的関数：回帰
              'metric': 'rmse',               # 分類モデルの性能を測る指標
              'learning_rate': 0.03,            # 学習率（初期値0.1）
#             learning_rate= 0.03,
#             'max_depth': 3,
#             'num_leaves': 10,
#             'feature_fraction':0.7,
#             'bagging_fraction':1,
#             'bagging_freq':20,
#             'n_jobs':5,        
#               'lambda_l1': 2, 
#               'max_depth' : 7
    }
    
    # 学習
    evaluation_results = {}                              # 学習の経過を保存する箱
    model = lgb.train(params,                            # 上記で設定したパラメータ
                      lgb_train,                         # 使用するデータセット
                      num_boost_round=10000000,              # 学習の回数
                      valid_names=['train', 'valid'],    # 学習経過で表示する名称
                      valid_sets=[lgb_train, lgb_eval],  # モデル検証のデータセット
                      evals_result=evaluation_results,   # 学習の経過を保存
                      early_stopping_rounds=20,          # アーリーストッピング
                      verbose_eval=0)                    # 学習の経過の非表示
    
    # 学習が終わったモデルをリストに保存
    models.append(model) 
    
    
    # 【ブロック３：　モデルで予測】
    # 学習したモデルで予測
    y_train_pred = model.predict(X_train_cv, num_iteration=model.best_iteration)
    y_eval_pred = model.predict(X_eval_cv, num_iteration=model.best_iteration)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    
    # 学習用データでの予測値をデータフレームに保存
    df_train_cv_pred = pd.DataFrame({round_no: y_train_pred},
                                     index=train_cv_no)
    df_train_preds = df_train_preds.join(df_train_cv_pred, how='left')
    
    # 検証用データでの予測値をデータフレームに保存
    df_eval_pred = pd.DataFrame({'y_eval': y_eval_cv,
                                 'y_eval_pred': y_eval_pred})
    df_eval_preds = df_eval_preds.append(df_eval_pred)    
    
    # テストデータでの予測値をデータフレームに保存
    df_test_cv_pred = pd.DataFrame({round_no: y_test_pred})
    df_test_preds = pd.concat([df_test_preds, df_test_cv_pred], axis=1)

    #  r2_score を計算
    train_score = r2_score(y_train_cv, y_train_pred)
    eval_score = r2_score(y_eval_cv, y_eval_pred)
    test_score = r2_score(y_test, y_test_pred)
    
    #  RMSE を計算
    train_RMSE_score = np.sqrt(mean_squared_error(y_train_cv, y_train_pred))
    eval_RMSE_score = np.sqrt(mean_squared_error(y_eval_cv, y_eval_pred))
    test_RMSE_score = np.sqrt(mean_squared_error(y_test, y_test_pred))
    
    # スコアを表示
    print('R^2 train: %.5f, eval: %.5f, test: %.5f' 
          % (train_score, eval_score, test_score),
         '  |  RMSE train: %.5f, eval: %.5f, test: %.5f' 
          % (train_RMSE_score, eval_RMSE_score, test_RMSE_score))
    
    # R2の保存
    df_R2_cv = pd.DataFrame({'train': [train_score],
                             'eval': [eval_score],
                             'test': [test_score]},
                             index=[round_no])
    df_R2 = df_R2.append(df_R2_cv)
    
    # RMSEの保存
    df_RMSE_cv =pd.DataFrame({'train': [train_RMSE_score],
                              'eval': [eval_RMSE_score],
                              'test': [test_RMSE_score]},
                              index=[round_no])
    df_RMSE = df_RMSE.append(df_RMSE_cv)
    
    # ラウンド数のカウンタを更新
    round_no += 1


# 保存したR2の平均値   
R2_ave = np.array(df_R2.mean()) #.to_numpy()

# 保存したRMSEの平均値   
RMSE_ave = np.array(df_RMSE.mean()) #.to_numpy()

# 平均値を表示
print('Average:')
print('R^2 train: %.5f, eval: %.5f, test: %.5f'
      % (R2_ave[0], R2_ave[1], R2_ave[2]),
      '  |  RMSE train: %.5f, eval: %.5f, test: %.5f'
      % (RMSE_ave[0], RMSE_ave[1], RMSE_ave[2]))



Requirement already up-to-date: pandas in /Users/madorin/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages (1.3.5)
Requirement not upgraded as not directly required: pytz>=2017.3 in /Users/madorin/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages (from pandas) (2018.5)
Requirement not upgraded as not directly required: numpy>=1.17.3; platform_machine != "aarch64" and platform_machine != "arm64" and python_version < "3.10" in /Users/madorin/.local/lib/python3.7/site-packages (from pandas) (1.19.5)
Requirement not upgraded as not directly required: python-dateutil>=2.7.3 in /Users/madorin/.pyenv/versions/anaconda3-5.3.1/lib/python3.7/site-packages (from pandas) (2.7.3)
Requirement not upgraded as not directly required: six>=1.5 in /Users/madorin/.local/lib/python3.7/site-packages (from python-dateutil>=2.7.3->pandas) (1.15.0)
conda 4.10.0 requires ruamel_yaml_conda>=0.11.14, which is not installed.
open3d 0.12.0+3483e4b has requirement pyyaml==5.4.1, but you'll have p